In [24]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

In [23]:
# Python Decorators Refresher
"""
By definition, a decorator is a function that takes another function 
and extends the behavior of the latter function without explicitly modifying it
"""
import time

def timer(func):
    def wrapper(*args, **kwargs):
        starttime = time.perf_counter()
        print(f"Calling {func.__name__}")
        value = func(*args, **kwargs)
        endtime = time.perf_counter()
        time_taken = endtime - starttime
        print(f"{func.__name__} took {time_taken:4f} seconds to process")
        return value
    
    return wrapper

@timer
def say_hello(name,age):
    return f"Hello {name}, your age is {age}."

say_hello("Saurabh",10)

Calling say_hello
say_hello took 0.000374 seconds to process


'Hello Saurabh, your age is 10.'

### @tool decorator
Tool Decorator makes tools out of functions

In [8]:
"""
This is 1 way of doing things
"""
@tool
def multiply(a: int, b:int)->int:
    "Multiply a & b"
    return a*b

@tool
def add(a:int, b:int)->int:
    "Add a & bs"
    return a+b

tools = [multiply,add]

In [33]:
"""
This is 2nd way of doing things
"""

from langchain_core.pydantic_v1 import BaseModel, Field

class Add(BaseModel):
    a:int=Field(...,description="First integer")
    b:int=Field(...,description="Second Integer")

class Multiply(BaseModel):
    a:int=Field(...,description="First integer")
    b:int=Field(...,description="Second Integer")

tools = [Add, Multiply]

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

llm_with_tools = llm.bind_tools(tools)

query = "What is 3*12? Also what is 3+3?"


# The role of LLM here is to just identify the right parameters and functions to call
# LLM is not called using below line of code

result = llm_with_tools.invoke(query)

print(result)
print("**"*50)
print(result.tool_calls)



# To make a call to LLM and get result of a query, we have to call LLM.invoke

print(f"LLM Result\n{llm.invoke(query)}")



content='' additional_kwargs={'tool_calls': [{'id': 'call_JBonyXD46fPCm48dcT1MArEA', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'Multiply'}, 'type': 'function'}, {'id': 'call_Gm8GOMP0BguwokTQzw2FdMHy', 'function': {'arguments': '{"a": 3, "b": 3}', 'name': 'Add'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 90, 'total_tokens': 140}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-faf99777-2b78-4e3f-97ae-39fb6bd9d7b1-0' tool_calls=[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_JBonyXD46fPCm48dcT1MArEA'}, {'name': 'Add', 'args': {'a': 3, 'b': 3}, 'id': 'call_Gm8GOMP0BguwokTQzw2FdMHy'}]
****************************************************************************************************
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_JBonyXD46fPCm48dcT1MArEA'}, {'name': 'Add', 'args': {'a': 3, 'b': 3}, 'id': 'call_G